In [2]:
import gym
import numpy as np

import tensorflow as tf
import sonnet as snt
import tensorflow_probability as tfp
tfd = tfp.distributions

tf.enable_eager_execution()

In [66]:
embedding_dim = 4
memory_dim = 8
n_memory_dims = 4

In [67]:
env = gym.make('CartPole-v0')

/home/act65/anaconda3/lib/python3.6/site-packages/gym/envs/registration.py:14: DeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [68]:
# [Cart Position, Cart Velocity, Pole Angle , Pole Velocity At Tip]
obs = env.reset()

In [69]:
encoder = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation=tf.nn.selu),
    tf.keras.layers.Dense(16, activation=tf.nn.selu),
    tf.keras.layers.Dense(embedding_dim),
])
# encode into a space where

In [70]:
vq_vae = snt.nets.VectorQuantizer(
  embedding_dim=embedding_dim,
  num_embeddings=30,
  commitment_cost=1)

In [73]:
h = encoder(tf.constant(obs, shape=[1, 4], dtype=tf.float32))
vq_results = vq_vae(h, True) 
# want to generalise to more dims
# also want to minimise its entropy. introduce more structure!
# needs to return a neighborhood. not just a single one?
z = vq_results['quantize']

In [88]:
 # extract neighborhoods
idx = vq_results['encoding_indices'][0]
neighbors = tf.nn.embedding_lookup(tf.transpose(vq_vae.embeddings), [idx-1, idx, idx+1])

In [92]:
energy_fn = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation=tf.nn.selu),
    tf.keras.layers.Dense(16, activation=tf.nn.selu),
    tf.keras.layers.Dense(1),
])

In [100]:
global_state = None
neighbors = fn(global_state)
energies = energy_fn(neighbors)

# linear comb from probabilities. or could sample. but it needs to be differentiable
dist = tfd.RelaxedOneHotCategorical(1.0, logits=energies)
next_state_idx = dist.sample()
next_state = tf.matmul(states, next_state_idx)

In [108]:
# 2 types of neighbor
# - the local energies
# - the neighboring states

# but these are related!?
# the likely neighboring states are determined by the local energies!?

In [107]:
# same energy fn is applied to all neighborhoods
# neighborhoods are indexed by current observation

# want: energy fn doesnt get access to the global state. 
# rather works locally. aka the global state is locally structured. 